In [1]:
! pip install gtts
! pip install playsound
! pip install gradio
! pip install transformers
! pip install torch

  Preparing metadata (setup.py) ... done
  Created wheel for playsound: filename=playsound-1.3.0-py3-none-any.whl size=7020 sha256=66c7bb833176eb1a191987fc63e86ea2b330f5502d176e78e2bc327f7c4e0785
  Stored in directory: /root/.cache/pip/wheels/90/89/ed/2d643f4226fc8c7c9156fc28abd8051e2d2c0de37ae51ac45c
Successfully built playsound
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.2/57.2 MB 11.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 320.4/320.4 kB 21.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 94.8/94.8 kB 8.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.2/11.2 MB 39.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 73.2/73.2 kB 2.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.3/62.3 kB 4.0 MB/s eta 0:00:00
  Attempting uninstall: markupsafe
    Found existing installation: MarkupSafe 3.0.2
    Uninstalling MarkupSafe-3.0.2:
      Successfully uninstalled MarkupSafe-3.0.2


# Import the Libraries

In [7]:
import gradio as gr
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import os
import torch
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM
from gtts import gTTS
import numpy as np
import shutil

class MultilingualExpensesTracker:
    def __init__(self, csv_path='/content/expenses_with_budget.csv'):
        self.csv_path = csv_path
        self.budget = None
        self.categories = [
            'Food', 'Transport', 'Bills', 'Entertainment', 'Education/Stationary',
            'Mobile Internet', 'Saving', 'Electricity', 'Water', 'Gas', 'Fuel',
            'Birthday', 'Rents', 'Household Items'
        ]
        self.reset_csv()

        self.load_translation_models()
        self.load_expenses()
        self.alert_path = '/content/budget_alert.mp3'

    def reset_csv(self):
        if os.path.exists(self.csv_path):
            os.remove(self.csv_path)
        empty_df = pd.DataFrame(columns=['date', 'category', 'amount'])
        empty_df.to_csv(self.csv_path, index=False)
        print("Expenses CSV reset.")

    def load_translation_models(self):
        try:
            self.en_to_urdu_tokenizer = AutoTokenizer.from_pretrained("Helsinki-NLP/opus-mt-en-ur")
            self.en_to_urdu_model = AutoModelForSeq2SeqLM.from_pretrained("Helsinki-NLP/opus-mt-en-ur")

            self.en_to_arabic_tokenizer = AutoTokenizer.from_pretrained("Helsinki-NLP/opus-mt-en-ar")
            self.en_to_arabic_model = AutoModelForSeq2SeqLM.from_pretrained("Helsinki-NLP/opus-mt-en-ar")

            self.en_to_hindi_tokenizer = AutoTokenizer.from_pretrained("Helsinki-NLP/opus-mt-en-hi")
            self.en_to_hindi_model = AutoModelForSeq2SeqLM.from_pretrained("Helsinki-NLP/opus-mt-en-hi")
        except Exception as e:
            print(f"Translation model loading error: {e}")
            self.en_to_urdu_model = None
            self.en_to_arabic_model = None
            self.en_to_hindi_model = None

    def load_expenses(self):
        try:
            self.expenses = pd.read_csv(self.csv_path)
            self.expenses['date'] = pd.to_datetime(self.expenses['date'])
        except FileNotFoundError:
            self.expenses = pd.DataFrame(columns=['date', 'category', 'amount'])

    def generate_comprehensive_visualizations(self):
        if self.expenses.empty:
            return None, None, None, None

        category_summary = self.expenses.groupby('category')['amount'].sum()
        monthly_expenses = self.expenses.groupby(pd.Grouper(key='date', freq='M'))['amount'].sum()

        # Pie Chart
        plt.figure(figsize=(10, 6))
        category_summary.plot(kind='pie', autopct='%1.1f%%', startangle=90)
        plt.title('Expense Distribution by Category')
        plt.ylabel('')
        pie_chart = plt.gcf()
        plt.close()

        # Line Chart
        plt.figure(figsize=(12, 6))
        monthly_expenses.plot(kind='line', marker='o')
        plt.title('Monthly Expense Trend')
        plt.xlabel('Month')
        plt.ylabel('Total Expenses (PKR)')
        plt.xticks(rotation=45)
        plt.tight_layout()
        trend_chart = plt.gcf()
        plt.close()

        # Bar Chart
        plt.figure(figsize=(12, 6))
        category_summary.plot(kind='bar', color='skyblue')
        plt.title('Expense Comparison Across Categories')
        plt.xlabel('Category')
        plt.ylabel('Total Expenses (PKR)')
        plt.xticks(rotation=45, ha='right')
        plt.tight_layout()
        category_bar_chart = plt.gcf()
        plt.close()

        # Boxplot
        plt.figure(figsize=(12, 6))
        category_expense_data = [self.expenses[self.expenses['category'] == cat]['amount'] for cat in self.categories]
        plt.boxplot(category_expense_data, labels=self.categories)
        plt.title('Expense Distribution by Category')
        plt.xlabel('Category')
        plt.ylabel('Expense Amount (PKR)')
        plt.xticks(rotation=45, ha='right')
        plt.tight_layout()
        boxplot = plt.gcf()
        plt.close()

        return pie_chart, trend_chart, category_bar_chart, boxplot

    def get_detailed_summary_statistics(self):
        if self.expenses.empty:
            return {}

        total_expense = self.expenses['amount'].sum()
        avg_expense = self.expenses['amount'].mean()
        median_expense = self.expenses['amount'].median()

        category_stats = self.expenses.groupby('category')['amount'].agg([
            ('Total', 'sum'), ('Average', 'mean'), ('Max', 'max'), ('Min', 'min')
        ])

        monthly_expenses = self.expenses.groupby(pd.Grouper(key='date', freq='M'))['amount'].sum()

        summary = {
            'Overall Statistics': {
                'Total Expenses': f"{total_expense:.2f} PKR",
                'Average Expense': f"{avg_expense:.2f} PKR",
                'Median Expense': f"{median_expense:.2f} PKR",
                'Number of Expenses': len(self.expenses),
                'Number of Categories': len(self.expenses['category'].unique())
            },
            'Monthly Insights': {
                'Average Monthly Expense': f"{monthly_expenses.mean():.2f} PKR",
                'Highest Month': f"{monthly_expenses.max():.2f} PKR",
                'Lowest Month': f"{monthly_expenses.min():.2f} PKR"
            },
            'Category Breakdown': category_stats.to_dict()
        }

        return summary

    def set_budget(self, budget):
        self.budget = budget
        return f"Budget set to {budget:.2f} PKR"

    def translate_text(self, text, target_lang='en'):
        if target_lang == 'urdu' and self.en_to_urdu_model:
            inputs = self.en_to_urdu_tokenizer(text, return_tensors="pt", padding=True)
            translation_ids = self.en_to_urdu_model.generate(**inputs)
            translated_text = self.en_to_urdu_tokenizer.batch_decode(translation_ids, skip_special_tokens=True)[0]
            return translated_text
        elif target_lang == 'arabic' and self.en_to_arabic_model:
            inputs = self.en_to_arabic_tokenizer(text, return_tensors="pt", padding=True)
            translation_ids = self.en_to_arabic_model.generate(**inputs)
            translated_text = self.en_to_arabic_tokenizer.batch_decode(translation_ids, skip_special_tokens=True)[0]
            return translated_text
        elif target_lang == 'hindi' and self.en_to_hindi_model:
            inputs = self.en_to_hindi_tokenizer(text, return_tensors="pt", padding=True)
            translation_ids = self.en_to_hindi_model.generate(**inputs)
            translated_text = self.en_to_hindi_tokenizer.batch_decode(translation_ids, skip_special_tokens=True)[0]
            return translated_text
        else:
            return text

    def add_expenses_multiple_categories(self, language, *amounts):
        expenses_to_add = []
        categories_without_saving = [cat for cat in self.categories if cat != 'Saving']
        for category, amount in zip(categories_without_saving, amounts):
            if amount > 0:
                expenses_to_add.append({'date': pd.Timestamp.now(), 'category': category, 'amount': amount})

        if not expenses_to_add:
            return "No expenses to add.", None, 0

        new_expenses_df = pd.DataFrame(expenses_to_add)
        self.expenses = pd.concat([self.expenses, new_expenses_df], ignore_index=True)
        self.expenses.to_csv(self.csv_path, index=False)

        total_spent = new_expenses_df['amount'].sum()
        remaining_budget = self.calculate_remaining_budget()

        if remaining_budget > 0:
            saving_amount = remaining_budget
            self.add_saving(saving_amount)
        else:
            saving_amount = 0

        status_message = self.translate_text(f"Expenses added successfully: {total_spent:.2f} PKR. ", language)

        audio = None
        if self.budget is not None and self.get_total_expenses() > self.budget:
            alert_text = self.translate_text("Budget exceeded!", language)
            tts = gTTS(text=alert_text, lang=self.get_lang_code(language))
            tts.save(self.alert_path)
            audio = self.alert_path
            status_message = f"{status_message} {self.translate_text('Budget Exceeded', language)}"
        return status_message, audio, saving_amount

    def add_saving(self, amount):
        saving_entry = {'date': pd.Timestamp.now(), 'category': 'Saving', 'amount': amount}
        self.expenses = pd.concat([self.expenses, pd.DataFrame([saving_entry])], ignore_index=True)
        self.expenses.to_csv(self.csv_path, index=False)

    def calculate_remaining_budget(self):
        if self.budget is None:
            return 0
        total_expenses = self.expenses['amount'].sum()
        return max(0, self.budget - total_expenses)

    def get_total_expenses(self):
        return self.expenses['amount'].sum() if not self.expenses.empty else 0

    def get_lang_code(self, language):
        if language == "urdu":
            return "ur"
        elif language == "arabic":
            return "ar"
        elif language == "hindi":
            return "hi"
        else:
            return 'en'


def create_budget_tracker_interface():
    tracker = MultilingualExpensesTracker()

    with gr.Blocks() as demo:
        gr.Markdown("# 🏦 Multilingual Budget & Expenses Tracker")

        with gr.Tabs():
            with gr.TabItem("Budget Setting"):
                with gr.Column():
                    gr.Markdown("## 💰 Set Budget")
                    budget_input = gr.Number(label="Set Your Monthly Budget (PKR)")
                    budget_status = gr.Textbox(label="Budget Status", interactive=False)
                    set_budget_btn = gr.Button("Set Budget")
                    set_budget_btn.click(
                        tracker.set_budget,
                        inputs=budget_input,
                        outputs=budget_status
                    )

            with gr.TabItem("Add Expenses"):
                with gr.Column():
                    gr.Markdown("## ➕ Add Expenses")
                    language_dropdown = gr.Dropdown(
                        label="Select Language",
                        choices=['en', 'urdu', 'arabic', 'hindi'],
                        value='en'
                    )

                    with gr.Row():
                        expense_inputs = []
                        for category in [cat for cat in tracker.categories if cat != 'Saving']:
                            expense_inputs.append(
                                gr.Number(label=f"{category} (PKR)", value=0)
                            )

                    saving_input = gr.Number(label="Saving (PKR)", value=0, interactive=False)

                    expense_status = gr.Textbox(label="Expense Status", interactive=False)
                    audio_alert = gr.Audio(label="Budget Alert", interactive=False)

                    add_expense_btn = gr.Button("Add Expenses")
                    add_expense_btn.click(
                        fn=tracker.add_expenses_multiple_categories,
                        inputs=[language_dropdown] + expense_inputs,
                        outputs=[expense_status, audio_alert, saving_input]
                    )

            with gr.TabItem("Visualizations"):
                gr.Markdown("## 📊 Expense Visualizations")
                with gr.Row():
                    pie_chart = gr.Plot(label="Category Distribution")
                    trend_chart = gr.Plot(label="Monthly Expense Trend")

                with gr.Row():
                    category_bar_chart = gr.Plot(label="Category Comparison")
                    boxplot = gr.Plot(label="Expense Distribution")

                visualize_btn = gr.Button("Generate Visualizations")
                visualize_btn.click(
                    fn=tracker.generate_comprehensive_visualizations,
                    outputs=[pie_chart, trend_chart, category_bar_chart, boxplot]
                )

            with gr.TabItem("Summary Statistics"):
                gr.Markdown("## 📝 Detailed Expense Analysis")
                stats_output = gr.JSON(label="Comprehensive Expense Statistics")
                stats_btn = gr.Button("Get Detailed Statistics")
                stats_btn.click(
                    fn=tracker.get_detailed_summary_statistics,
                    outputs=stats_output
                )

    return demo

if __name__ == "__main__":
    os.makedirs('/content', exist_ok=True)
    tracker_interface = create_budget_tracker_interface()
    tracker_interface.launch(share=True)

Expenses CSV reset.


/usr/local/lib/python3.10/dist-packages/transformers/models/marian/tokenization_marian.py:175: UserWarning: Recommended: pip install sacremoses.
  warnings.warn("Recommended: pip install sacremoses.")


Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://a10f154c7d5642e5b7.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
